In [ ]:
import os
import numpy as np
import pandas as pd
import pickle as pkl
from gcp_utils import constants
from gcp_utils.tools.utils import format_as_json
from firebase_admin import firestore, initialize_app
from database_tools.tools.dataset import ConfigMapper
from gcp_utils.tools import preprocess, predict, utils
from plotly.subplots import make_subplots
from alive_progress import alive_bar

os.chdir('/home/cam/Documents/gcp_utils/')

pd.options.plotting.backend = 'plotly'

initialize_app()
database = firestore.client()
col = database.collection(u'bpm_data_test').document('wx1jF08b3DTPijtQcwGiEwpEFai2').collection('frames')

In [ ]:
cm = ConfigMapper(constants.CONFIG_PATH)

processed_frames = []
processed_windows = []

with alive_bar(force_tty=True, title_length=25) as bar:
    for i, doc in enumerate(col.stream()):
        bar.title('processing frame...')
        frame_data = doc.to_dict()

        processed = preprocess.process_frame(frame_data['red_frame'], frame_data['ir_frame'], cm=cm)
        cardiac_metrics = predict.predict_cardiac_metrics(
            red=processed['red_frame_for_processing'],
            ir=processed['ir_frame_for_processing'],
            cm=cm,
        )
        windows = [s for s in utils.generate_window_document(processed['windows'], frame_data['fid'])]
        frame_data.update({
            'status': 'processed',
            'red_frame_for_processing': processed['red_frame_for_processing'],
            'ir_frame_for_processing': processed['ir_frame_for_processing'],
            'red_frame_for_presentation': processed['red_frame_for_presentation'],
            'ir_frame_for_presentation': processed['ir_frame_for_presentation'],
            'combined_frame_for_processing': processed['combined_frame_for_processing'],
            'combined_frame_for_presentation': processed['combined_frame_for_presentation'],
            'pulse_rate': cardiac_metrics['pulse_rate'],
            'spo2': cardiac_metrics['spo2'],
            'r': cardiac_metrics['r'],
        })
        processed_frames.append(frame_data)

        for w in windows:
            bar.title('validating window...')
            window_data = w
            result = preprocess.validate_window(window_data['ppg'], cm, force_valid=True)
            window_data.update({
                'status': result['status'],
                'vpg': result['vpg'],
                'apg': result['apg'],
                'ppg_scaled': result['ppg_scaled'],
                'vpg_scaled': result['vpg_scaled'],
                'apg_scaled': result['apg_scaled'],
                'f0': result['f0'],
                'snr': result['snr'],
                'beat_sim': result['beat_sim'],
            })

            if result['status'] == 'valid':
                bar.title('predicting...')
                result = predict.predict_bp(format_as_json(window_data)[0], cm)
                window_data.update({
                    'status': 'predicted',
                    'abp': result['abp'],
                })
            processed_windows.append(window_data)
            bar()

In [ ]:
frame = [x['red_frame'] for x in processed_frames if x['status'] == 'processed']
n = len(frame)

fig = make_subplots(rows=n, cols=1)
for i, sig in enumerate(frame):
    fig.add_scatter(y=sig, row=i+1, col=1)

fig.update_layout(
    height=4000
)

In [ ]:
ppg = [x['ppg'] for x in processed_windows if x['status'] == 'predicted']
n = len(ppg)

fig = make_subplots(rows=n, cols=2)
for i, sig in enumerate(ppg):
    fig.add_scatter(y=sig, row=i+1, col=1, name=f'ppg_{i}')

abp = [x['abp'] for x in processed_windows if x['status'] == 'predicted']
n = len(abp)

for i, sig in enumerate(abp):
    fig.add_scatter(y=sig, row=i+1, col=2, name=f'pred_{i}')

fig.update_layout(
    width=1200,
    height=10000
)

In [ ]:
x = np.array([x['ppg'] for x in processed_windows if np.max(x['ppg']) < 10000])
min_ = np.min(x)
max_ = np.max(x)

# min_ = 0
# max_ = 5600

# with open('../gcp_utils/bpm_scaler.pkl', 'wb') as f:
#     pkl.dump((min_, max_), f)

min_, max_

In [ ]:
_ = [print(x['snr']) for x in processed_windows]

In [ ]:
_ = [print(x['f0']) for x in processed_windows]

In [ ]:
_ = [print(x['beat_sim']) for x in processed_windows]

In [ ]:
_ = [print(x['notches']) for x in processed_windows]

In [ ]:
_ = [print(x['flat_lines']) for x in processed_windows]